In [1]:
from scipy.stats import norm
import numpy as np
import matplotlib.pyplot as plt

from pathlib import Path
from astropy.nddata import CCDData
from astropy.io import fits
from ccdproc import ImageFileCollection
import ccdproc as ccdp

from astropy.nddata import CCDData
from astropy.stats import mad_std
import astropy.units as u
from convenience_functions import show_image

# suppress warnings is not recommended :I
import warnings
warnings.filterwarnings("ignore")

In [2]:
bias_dir = './data/bias/'
dark_dir = './data/dark/'
flat_dir = './data/flat/'

combined_bias = CCDData.read(bias_dir + 'combined_bias.fits')
combined_dark = CCDData.read(dark_dir + 'combined_dark_900.000.fits')
combined_V_flat = CCDData.read(flat_dir + 'combined_V_flat.fits')

def process_images(raw_dir, reduced_dir):
    raw_images = ImageFileCollection(raw_dir)
    raw_images = raw_images.files_filtered(include_path=True)
    
    for file_name in raw_images:
        raw_data = CCDData.read(file_name, unit='adu')
        reduced = ccdp.subtract_bias(raw_data, combined_bias)
        reduced = ccdp.subtract_dark(
            reduced,
            combined_dark,
            exposure_time='exptime',
            exposure_unit=u.second,
            scale=True
        )
        reduced = ccdp.flat_correct(reduced, combined_V_flat)
        reduced.data = reduced.data - np.median(reduced.data)
        name = file_name.split('/')[-1]
        reduced.write(reduced_dir + f'REDUCED_{name}', overwrite=True)

In [3]:
raw_dir = './data/tz_boo/raw/'
reduced_dir = './data/tz_boo/reduced/'

process_images(raw_dir, reduced_dir)

In [4]:
raw_dir = './data/cc_com/raw/'
reduced_dir = './data/cc_com/reduced/'

process_images(raw_dir, reduced_dir)